## Notebook 1: PDF 预处理

在这一系列中，我们将使用开源模型将PDF转换为播客。

获取播客的第一步是找到一个脚本，目前我们的步骤如下：
- 使用任意主题的PDF
- 使用 [mlx-community/Qwen2.5-3B-Instruct-4bit](https://huggingface.co/mlx-community/Qwen2.5-3B-Instruct-4bit) 模型，将其处理成文本文件。英文用的是1.5B，中文使用1.5B效果不太好。
- 在下一个 Notebook 中将其改写为播客稿件。

在这个 Notebook 中，我们将上传一个PDF，使用 `PyPDF2` 将其保存为 `.txt` 文件，后续我们会用轻量模型处理文本。

务必记得安装此项目依赖，不然跑不起来～🙂‍↔️

In [16]:
#!pip install -r requirements.txt

这里设置需要处理的PDF文件路径。

另外，如果你想要发挥GPU极致性能，求你更换大参数的模型，虽然轻量级模型对于此任务也能胜任。

In [17]:
pdf_path = '../resources/2402.13116v4.pdf'
DEFAULT_MODEL = "mlx-community/Qwen2.5-3B-Instruct-4bit"

In [18]:
import os
import warnings
from typing import Optional

import PyPDF2
from mlx_lm import load, generate
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

这里会检查文件有没有什么毛病～

In [19]:
def validate_pdf(file_path: str) -> bool:
    if not os.path.exists(file_path):
        print(f"Error: 此文件找不到啊🤔: {file_path}")
        return False
    if not file_path.lower().endswith('.pdf'):
        print("Error: 此文件不是 PDF 😵‍💫")
        return False
    return True

这里就简单的读取 PDF 保存为`.txt`文件。默认的最大字符数是10万，如果你有长篇大论要处理，随时可以调整哦！不过，记得要考虑模型的长度限制～

In [20]:
def extract_text_from_pdf(file_path: str, max_chars: int = 100000) -> Optional[str]:
    if not validate_pdf(file_path):
        return None

    try:
        with open(file_path, 'rb') as file:
            # 创建 PDF reader 对象
            pdf_reader = PyPDF2.PdfReader(file)

            # 获取总页数
            num_pages = len(pdf_reader.pages)
            print(f"处理 {num_pages} 页的 PDF...")

            extracted_text = []
            total_chars = 0

            # 遍历所有页面
            for page_num in range(num_pages):
                # 提取当前页面的文本
                page = pdf_reader.pages[page_num]
                text = page.extract_text()

                # 检查文本是否超过最大字符数
                if total_chars + len(text) > max_chars:
                    # 仅截取到最大字符数
                    remaining_chars = max_chars - total_chars
                    extracted_text.append(text[:remaining_chars])
                    print(f"在第 {page_num + 1} 页超过 {max_chars} 字符限制啦！！")
                    break

                extracted_text.append(text)
                total_chars += len(text)
                print(f"已处理 {page_num + 1}/{num_pages} 页")

            final_text = '\n'.join(extracted_text)
            print(f"\n提取完成！总字数：{len(final_text)}")
            return final_text

    except PyPDF2.PdfReadError:
        print("Error: 这 PDF 有毛病啊，请仔细看看～")
        return None
    except Exception as e:
        print(f"完了，挂了: {str(e)}")
        return None


获取 PDF 元信息

In [21]:
# 获取 PDF 元信息
def get_pdf_metadata(file_path: str) -> Optional[dict]:
    if not validate_pdf(file_path):
        return None

    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            metadata = {
                'num_pages': len(pdf_reader.pages),
                'metadata': pdf_reader.metadata
            }
            return metadata
    except Exception as e:
        print(f"提取元信息失败: {str(e)}")
        return None

Finally, we can run our logic to extract the details from the file

In [22]:
# 提取元信息
print("正在提取元信息...")
metadata = get_pdf_metadata(pdf_path)
if metadata:
    print("\nPDF 元信息:")
    print(f"页数: {metadata['num_pages']}")
    print("文档信息:")
    for key, value in metadata['metadata'].items():
        print(f"{key}: {value}")

# 提取文本
print("\n提取文本...")
extracted_text = extract_text_from_pdf(pdf_path)

# 预览前500字符文本
if extracted_text:
    print("\n预览提取后的文本 (前500字符):")
    print("-" * 50)
    print(extracted_text[:500])
    print("-" * 50)
    print(f"\n提取的总字符数: {len(extracted_text)}")

# 将提取的文本保存到 txt 文件中
if extracted_text:
    output_file = './resources/extracted_text.txt'
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(extracted_text)
    print(f"\n提取的文本已保存到 {output_file}")

正在提取元信息...

PDF 元信息:
页数: 43
文档信息:
/Author: 
/CreationDate: D:20241022021202Z
/Creator: LaTeX with hyperref
/Keywords: 
/ModDate: D:20241022021202Z
/PTEX.Fullbanner: This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5
/Producer: pdfTeX-1.40.25
/Subject: 
/Title: 
/Trapped: /False

提取文本...
处理 43 页的 PDF...
已处理 1/43 页
已处理 2/43 页
已处理 3/43 页
已处理 4/43 页
已处理 5/43 页
已处理 6/43 页
已处理 7/43 页
已处理 8/43 页
已处理 9/43 页
已处理 10/43 页
已处理 11/43 页
已处理 12/43 页
已处理 13/43 页
已处理 14/43 页
已处理 15/43 页
已处理 16/43 页
在第 17 页超过 100000 字符限制啦！！

提取完成！总字数：100016

预览提取后的文本 (前500字符):
--------------------------------------------------
1
A Survey on Knowledge Distillation of Large
Language Models
Xiaohan Xu1, Ming Li2, Chongyang Tao3, Tao Shen4, Reynold Cheng1, Jinyang Li1,
Can Xu5, Dacheng Tao6, Tianyi Zhou2
1The University of Hong Kong2University of Maryland3Microsoft
4University of Technology Sydney5Peking University6The University of Sydney
{shawnxxh,chongyangtao,hishentao }@gmail.com {min

### 预处理
现在我们来说说为啥不用正则表达式的提取，而使用大型语言模型的：
目前，我们已经从 PDF 中提取出一个文本文件。一般来说，由于字符、格式、Latex、表格等原因，PDF 提取出的内容可能会很凌乱。
解决这个问题的一种方法是使用正则表达式，但我们也可以使用轻量模型帮助清洗文本。
你可以试试修改 `SYS_PROMPT`

In [23]:
SYS_PROMPT = """您是一流的文本预处理器。以下是来自 PDF 的原始数据，请解析并以清晰可用的方式返回给播客作者。
原始数据中包含换行符、Latex 数学公式以及一些可以完全删除的冗余内容。请去掉对播客作者记录无用的信息。
不论主题如何，以上问题并不详尽，因此请聪明选择要删除的信息，发挥创造力。
请注意，您的任务仅限于清理文本和必要时重写，在删除细节时要果断。当您进行文本处理时，请确保输出中文。
如果遇到英文或其他语言，请翻译为中文后再输出。
请勿添加 Markdown 格式化或特殊字符。
一开始便直接开始响应处理后的文本，不做任何确认，谢谢！
这里是文本：
"""

为了避免模型一次处理整个文件，我们将分块处理文件。

In [24]:
def create_word_bounded_chunks(text, target_chunk_size):
    """
    这里会根据 target_chunk_size 分块
    """
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        word_length = len(word) + 1  # +1 for the space
        if current_length + word_length > target_chunk_size and current_chunk:
            # Join the current chunk and add it to chunks
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_length = word_length
        else:
            current_chunk.append(word)
            current_length += word_length

    # Add the last chunk if it exists
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

让我们加载模型并开始处理文本块吧！✨

In [25]:
model, tokenizer = load(DEFAULT_MODEL)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [26]:
def process_chunk(text_chunk, chunk_num):
    """处理文本块，并返回模型处理好的文本"""
    conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": text_chunk},
    ]

    prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    processed_text = generate(
        model=model,
        tokenizer=tokenizer,
        prompt=prompt,
        max_tokens=512,
        temp=0.7
    )

    # Print chunk information for monitoring
    #print(f"\n{'='*40} Chunk {chunk_num} {'='*40}")
    print(f"INPUT TEXT:\n{text_chunk[:500]}...")  # Show first 500 chars of input
    print(f"\nPROCESSED TEXT:\n{processed_text[:500]}...")  # Show first 500 chars of output
    print(f"{'=' * 90}\n")

    return processed_text

In [27]:
INPUT_FILE = "./resources/extracted_text.txt"  
CHUNK_SIZE = 1000

# 读取文件
with open(INPUT_FILE, 'r', encoding='utf-8') as file:
    text = file.read()

chunks = create_word_bounded_chunks(extracted_text, CHUNK_SIZE)
num_chunks = len(chunks)

In [28]:
num_chunks

101

In [29]:
# 创建输出文件路径
output_file = f"./resources/clean_{os.path.basename(INPUT_FILE)}"

processed_text = ""

In [30]:
with open(output_file, 'w', encoding='utf-8') as out_file:
    for chunk_num, chunk in enumerate(tqdm(chunks, desc="Processing chunks")):
        # 处理文本块
        processed_chunk = process_chunk(chunk, chunk_num)
        processed_text += processed_chunk + "\n"

        # 将处理后的文本写入输出文件
        out_file.write(processed_chunk + "\n")
        out_file.flush()

Processing chunks:   0%|          | 0/101 [00:00<?, ?it/s]

INPUT TEXT:
1 A Survey on Knowledge Distillation of Large Language Models Xiaohan Xu1, Ming Li2, Chongyang Tao3, Tao Shen4, Reynold Cheng1, Jinyang Li1, Can Xu5, Dacheng Tao6, Tianyi Zhou2 1The University of Hong Kong2University of Maryland3Microsoft 4University of Technology Sydney5Peking University6The University of Sydney {shawnxxh,chongyangtao,hishentao }@gmail.com {minglii,tianyi }@umd.edu ckcheng@cs.hku.hk jl0725@connect.hku.hk Abstract —In the era of Large Language Models (LLMs), Knowledge Distillati...

PROCESSED TEXT:
在大规模语言模型（LLMs）的时代，知识蒸馏（KD）作为将顶级专有LLM如GPT-4等先进能力转移给开源模型如LLaMa和Mistral的关键方法出现了。随着开源模型的蓬勃发展，KD在压缩这些模型和通过自身进行自我提升方面扮演了重要角色。本文全面回顾了KD在LLM领域中的作用，强调了其在赋予模型知识方面的重要功能。...

INPUT TEXT:
advanced knowledge to smaller models and its utility in model compression and self- improvement. Our survey is meticulously structured around three foundational pillars: algorithm ,skill, and verticalization – providing a comprehensive examination of KD mechanisms, the enhancement of specifi

让我们看看最后处理的效果吧～🍻

In [31]:
print(f"\n处理完成！")
print(f"输入文件: {INPUT_FILE}")
print(f"输出文件: {output_file}")
print(f"已处理总文本块: {num_chunks}")

# 预览处理后的文本的开头和结尾。
print("\n预览处理后的文本：")
print("\n开头:")
print(processed_text[:1000])
print("\n...\n\n结尾:")
print(processed_text[-1000:])


处理完成！
输入文件: ./resources/extracted_text.txt
输出文件: ./resources/clean_extracted_text.txt
已处理总文本块: 101

预览处理后的文本：

开头:
在大规模语言模型（LLMs）的时代，知识蒸馏（KD）作为将顶级专有LLM如GPT-4等先进能力转移给开源模型如LLaMa和Mistral的关键方法出现了。随着开源模型的蓬勃发展，KD在压缩这些模型和通过自身进行自我提升方面扮演了重要角色。本文全面回顾了KD在LLM领域中的作用，强调了其在赋予模型知识方面的重要功能。
通过将高级知识传授给较小的模型，并将其应用于模型压缩和自我提升方面具有实用价值。我们的调查围绕三个基础支柱展开：算法、技能和垂直化，提供了一个全面的评估知识传授机制、特定认知能力的提升及其在多个领域中的实际影响。至关重要的是，调查探讨了数据增强（DA）与知识传授（KD）之间的相互作用，展示了如何将DA作为KD框架中的强大范式来增强模型性能。通过利用DA生成富含上下文且特定技能的训练数据，知识传授超越了传统界限，使得开源模型能够模仿其专用版本的上下文敏感性、伦理一致性以及深层语义洞察。本文旨在为研究人员和实践者提供有价值的指南，详细介绍当前在知识传授方面的研究方法。
本文通过将专有和开源大型语言模型（LLMs）之间的差距进行桥梁连接，强调了更具访问性、效率和强大性的AI解决方案的潜在价值。我们强烈倡导遵守用于LLMs的法律条款，确保知识蒸馏（KD）的伦理和合法性应用。有关知识蒸馏的GitHub存储库可在https://github.com/Tebmer/Awesome-Knowledge-Distillation-of-LLMs处获取。关键术语——大型语言模型，知识蒸馏，数据增强，技能蒸馏，监督微调。1 引言 在人工智能（AI）不断发展的背景下，GPT-3.5（Ouyang et al., 2022）、GPT-4（OpenAI et al., 2023）、Gemini（Team et al., 2023）和Claude2等专有大型语言模型（LLMs）已经成为颠覆性的技术，极大地改变了我们对自然语言处理（NLP）的理解。这些模型以其庞大的规模而著称。
这些大型语言模型（LLMs）在解锁新的可能性方面具有核心意义。它们不仅能够生成类

### 下一个 Notebook: 转录员

现在我们已经预处理好文本，在下一个 Notebook 中将其转换为讲稿

In [32]:
#fin